In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import load_model

In [2]:
model = load_model('model/my_cifar10_model.keras', compile=False)

2024-09-28 15:33:24.359200: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Max
2024-09-28 15:33:24.359236: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 48.00 GB
2024-09-28 15:33:24.359241: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 18.00 GB
2024-09-28 15:33:24.359462: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-09-28 15:33:24.359478: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
sampleSubmission = pd.read_csv('data/sampleSubmission.csv')
sampleSubmission

,id,label
0,1,cat
1,2,cat
2,3,cat
3,4,cat
4,5,cat
...,...,...
299995,299996,cat
299996,299997,cat
299997,299998,cat
299998,299999,cat


In [4]:
image_dir = 'data/test'
image_paths = [f'{image_dir}/{id}.png' for id in sampleSubmission['id'].values]

In [5]:
def load_and_preprocess_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, [32, 32])
    return image

preprocessed_image = [load_and_preprocess_image(image_path) for image_path in image_paths]

preprocessed_image = tf.stack(preprocessed_image)

In [15]:
predictions = model.predict(preprocessed_image)

predicted_classes = np.argmax(predictions, axis=1)

predicted_classes

9375/9375 ━━━━━━━━━━━━━━━━━━━━ 9s 929us/step


array([4, 0, 1, ..., 4, 5, 1])

In [16]:
labels_name = pd.read_csv('data/labels.csv')
prediction_df = pd.DataFrame({
    'id' : predicted_classes
})
results = pd.merge(prediction_df, labels_name, on='id', how='inner')
results

,id,name
0,4,deer
1,0,airplane
2,1,automobile
3,8,ship
4,2,bird
...,...,...
299995,9,truck
299996,4,deer
299997,4,deer
299998,5,dog


In [22]:
sampleSubmission['label'] = results['name']
sampleSubmission.to_csv('data/sampleSubmission.csv', index=False)
sampleSubmission

,id,label
0,1,deer
1,2,airplane
2,3,automobile
3,4,ship
4,5,bird
...,...,...
299995,299996,truck
299996,299997,deer
299997,299998,deer
299998,299999,dog
